# SQL

Nous utiliserons le dialecte Transact-SQL (T-SQL) : [https://docs.microsoft.com/fr-fr/sql/t-sql/language-reference?view=sql-server-ver15](https://docs.microsoft.com/fr-fr/sql/t-sql/language-reference?view=sql-server-ver15)  

## Création BDD

In [ ]:
CREATE DATABASE [ImdbLike]

GO

USE [ImdbLike]

## Schéma

Un SGBD =\> plusieurs BDD

Une BDD =\> Plusieurs schémas

```
SELECT [...] FROM [data].[Table]
```

Ici, `[data]` est le schéma de `[Table]`

Les schémas servent à regrouper des tables de même nature =\> on peut y affecter des droits (ex: `[ref]` en RW pour l'admin mais lecture seule pour app, `[data]` en RW pour admin et app mais lecture seule pour le service compta)

In [ ]:
CREATE SCHEMA [ref]

GO

CREATE SCHEMA [data]

GO

## Création de table

In [ ]:
CREATE TABLE [ref].[Genre_GRE] -- [ref] car lecture-seule pour app
(
    [Id] INT IDENTITY(1, 1) PRIMARY KEY,
    [Nom] VARCHAR(25) NOT NULL
)

CREATE TABLE [data].[Film_FLM] -- [data] car lecture-écriture pour app
(
    [Id] BIGINT IDENTITY(1, 1),-- PRIMARY KEY, -- [Id] entier 64bits auto-incrément à partir de 1 par pas de 1, clé primaire
    [Nom] NVARCHAR(80) NOT NULL, -- NVARCHAR gère l'encode, VARCHAR non -> prend 2x plus de place
    [Descriptif] NVARCHAR(MAX) NOT NULL, -- Equivalent à TEXT -> remplacer TEXT par NVARCHAR
    -- [Genre] VARCHAR(20) NOT NULL -> Non, on normalise
    [GRE_Id] INT NOT NULL-- FOREIGN KEY REFERENCES [ref].[Genre_GRE]([Id])
    CONSTRAINT PK_FLM PRIMARY KEY ([Id]) -- On privilégie cette syntaxe -> Contrainte nommée + clé composite
    CONSTRAINT FK_FLM_GRE FOREIGN KEY ([GRE_Id]) REFERENCES [ref].[Genre_GRE]([Id])
)

CREATE TABLE [data].[Acteur_ACT] (
    [Id] UNIQUEIDENTIFIER NOT NULL DEFAULT NEWID() PRIMARY KEY, -- Id 128bits sous forme de chaîne
    [Nom] NVARCHAR(80) NOT NULL,
    [Prenom] NVARCHAR(80) NOT NULL
)

CREATE TABLE [data].[FilmActeur_FAC] (
    [FLM_Id] BIGINT NOT NULL,
    [ACT_Id] UNIQUEIDENTIFIER NOT NULL
    CONSTRAINT PK_FilmActeur PRIMARY KEY ([FLM_Id], [ACT_Id]),
    CONSTRAINT FK_FilmActeur_Acteur FOREIGN KEY ([ACT_Id]) REFERENCES [data].[Acteur_ACT]([Id]),
    CONSTRAINT FK_FilmActeur_Film FOREIGN KEY ([FLM_Id]) REFERENCES [data].[Film_FLM]([Id])
)


In [ ]:
SELECT NEWID()

## Modification de table

### Exercice

Modéliser des sociétés de production (ex: Lucas Film), les associées aux films (vous aurez besoin de l'instruction [`ALTER TABLE`](https://docs.microsoft.com/fr-fr/sql/t-sql/statements/alter-table-transact-sql?view=sql-server-ver15)). On partira du principe que un film possède une seule société de prod.

In [ ]:
CREATE TABLE [data].[SocieteProd_SPR] (
    [Id] BIGINT IDENTITY(1, 1) PRIMARY KEY,
    [Nom] NVARCHAR(100) NOT NULL
)

GO

ALTER TABLE [data].[Film_FLM]
    ADD [SPR_Id] BIGINT NOT NULL
    CONSTRAINT FK_FML_SPR FOREIGN KEY ([SPR_Id]) REFERENCES [data].[SocieteProd_SPR]([Id])

GO


## Opérations CRUD

Create, Read, Update, Delete

  

Create -\> INSERT

Read -\> SELECT

Update -\> UPDATE

Delete -\> DELETE

In [ ]:
INSERT INTO [ref].[Genre_GRE] ([Nom])
VALUES ('Science Fiction'), ('Action'), ('Aventure')

INSERT INTO [data].[SocieteProd_SPR] ([Nom])
VALUES (N'Lucas Film'), (N'Gaumont')

INSERT INTO [data].[Acteur_ACT] ([Nom], [Prenom])
VALUES (N'Ford', N'Harrisson'), (N'Depp', N'Johny')

In [ ]:
INSERT INTO [data].[Film_FLM] ([Nom], [Descriptif], [GRE_Id], [SPR_Id])
VALUES
    (N'Indiana Jones', N'Descriptif du film', 3, 1),
    (N'Pirates de Caraïbes', N'Descriptif de l''autre film', 2, 2)

In [ ]:
INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 1
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Ford'

INSERT INTO [data].[FilmActeur_FAC] ([ACT_Id], [FLM_Id])
SELECT TOP 1 [Id], 2
FROM [data].[Acteur_ACT]
WHERE [Nom] = 'Depp'

In [ ]:
UPDATE [data].[Film_FLM]
SET [Descriptif] = N'Film sur un archéologue'
WHERE [Id] = 1

UPDATE [data].[Film_FLM]
SET [Descriptif] = N'Film de pirates'
WHERE [Id] = 2


In [ ]:
DELETE FROM [ref].[Genre_GRE]
WHERE [Id] = 1

## Vues

Reppel jointures : https://sql.sh/cours/jointures

In [ ]:
SELECT [FLM].[Id], [FLM].[Nom], [GRE].[Nom] AS [Genre], [SPR].[Nom] AS [Société de prod]
FROM [data].[Film_FLM] AS [FLM]
INNER JOIN [ref].[Genre_GRE] AS [GRE] ON [FLM].[GRE_Id] = [GRE].[Id]
INNER JOIN [data].[SocieteProd_SPR] AS [SPR] ON [FLM].[SPR_Id] = [SPR].[Id]

In [ ]:
CREATE OR ALTER VIEW [data].[FicheFilm]
AS
SELECT [FLM].[Id], [FLM].[Nom], [GRE].[Nom] AS [Genre], [SPR].[Nom] AS [Société de prod]
FROM [data].[Film_FLM] AS [FLM]
INNER JOIN [ref].[Genre_GRE] AS [GRE] ON [FLM].[GRE_Id] = [GRE].[Id]
INNER JOIN [data].[SocieteProd_SPR] AS [SPR] ON [FLM].[SPR_Id] = [SPR].[Id]

In [ ]:
SELECT [Id], [Nom], [Genre], [Société de prod]
FROM [data].[FicheFilm]

### Exercice

1\. Insérer de nouveaux acteurs (4), genres, sociétés de prod (3) et de nouveaux films (5) dans vos tables (assignez des acteurs aux films).

2\. Créer une vue qui sort les acteurs avec leurs films associés (Nom de l'acteur, nom du film, genre du film, société de prod)